# Heroes of Pymoli - Data Analysis

##### Summary Notes
###### Who plays Heroes of Pymoli?
Heroes of Pymoli has 576 players. Of these players 84.03% of the players are male. 
The biggest age group in this game is 20-24, which had 44.8% of the players (258 players).
Young men are the most avide players of the game.

###### Who spends the most money on Heroes of Pymoli?
The biggest proportion of revenue comes from players in the 20-24 group. 
Although their average purchase value per person is lower than in other age categories,
they account for the most purchses.

Although, the 35-39 age group is smaller and accounts for fewer purchases, their average
purchase value per person is the highest. This indicates that players in this age group 
are more likely to make big purchases than players in other categories. 

###### What sells in Heroes of Pymoli?

The top selling item in Heroes of Pymoli is "Final Critic". Because there are so many items
available in the game, no one item accounts for a very large portion of the revenue. This suggests
that sales are driven specific player wants and needs. Though, the a popular item that
sells consistently suggests a popular configuration of class build and items that may be especially
beneficial to certain types of players. Though, further data would be required to identify these patterns.

In [31]:
# Import Dependencies
import os
import pandas as pd
import csv

In [32]:
# Create path for raw data file.
data_path = os.path.join(".","Resources","purchase_data.csv")

In [33]:
# Read Data into dataframe.
purchase_df = pd.read_csv(data_path)

# Summary Calculations

### Number of Unique Players

In [34]:
## Calculate Number of Unique Players
sn = purchase_df["SN"].unique()
player_count = len(sn)
print(player_count)

576


### Number of Unique Items

In [35]:
## Calculate Number of Unique Items
itemid = purchase_df["Item ID"].unique()
item_count = len(itemid)
print(item_count)

179


### Average Purchase Price

In [36]:
## Calculate Average Purchase Price
avg_price = purchase_df["Price"].mean()
print(avg_price)

3.050987179487176


### Total Purchases

In [37]:
# Calculate Total Purchases
purchase_id = purchase_df["Purchase ID"]
purchase_count = len(purchase_id)
print(purchase_count)

780


### Total Revenue 

In [38]:
## Calculate Total Revenue
price_df = purchase_df["Price"]
total_revenue = price_df.sum()
print(total_revenue)

2379.77


## Gender Demographics

In [56]:
# Creating Gender Aggregate
gender_agg = purchase_df.groupby("Gender")
gender_count = pd.DataFrame(gender_agg.nunique())


In [57]:
# Counts and Percentages by Gender
gender_count["Percentage"] = 100*gender_count["SN"]/player_count
gender_count["Percentage"] = gender_count["Percentage"].map("{:.4}%".format)
gender_count = gender_count[['SN','Percentage']].sort_values("Percentage",ascending = False).rename(columns = {"SN":"Player Count"})
gender_count

,Player Count,Percentage
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis - Gender


In [22]:
# create aggregate metrics by gender
gender_mean = pd.DataFrame(gender_agg.mean()["Price"].map("${:.3}".format)).rename(columns = {"Price":"Average Purchase Value"})
gender_sum = pd.DataFrame(gender_agg.sum()["Price"])
gender_sales = pd.DataFrame(gender_agg.sum()["Price"].map("${:.7}".format)).rename(columns = {"Price":"Total Sales"})
gender_count2 = pd.DataFrame(gender_agg.count()["Purchase ID"]).rename(columns = {"Purchase ID":"Purchase Count"})

# calculate average perchase per player and format to dollars
a = pd.DataFrame(gender_sum["Price"].astype(float)) 
b = pd.DataFrame(gender_count["Player Count"].astype(float))
gender_mean["Avg Purchase Value Per Person"] = a["Price"]/b["Player Count"]
gender_mean["Avg Purchase Value Per Person"] = gender_mean["Avg Purchase Value Per Person"].map("${:.3}".format)

#merge aggregate metrics 
gender_join = gender_count2.merge(gender_mean, on = 'Gender').merge(gender_sales, on = 'Gender')

#display
gender_join

,Purchase Count,Average Purchase Value,Avg Purchase Value Per Person,Total Sales
Gender,,,,
Female,113,$3.2,$4.47,$361.94
Male,652,$3.02,$4.07,$1967.64
Other / Non-Disclosed,15,$3.35,$4.56,$50.19


## Age Demographics

In [12]:
# Age Binning
age_bounds = [0,9,14,19,24,29,34,39,44,49]
age_names = ['Below 10','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49']
purchase_df["Age Group"] = pd.cut(purchase_df["Age"], age_bounds, labels=age_names)

In [13]:
# calculating aggregate metrics by age group,
age_agg = purchase_df.groupby("Age Group")
age_count = pd.DataFrame(age_agg.nunique()).rename(columns = {"SN":"Player Count"})
age_count["Percentage"] = 100*age_count["Player Count"]/player_count
age_count["Percentage"] = age_count["Percentage"].map("{:.3}%".format)
age_breakdown = age_count[["Player Count","Percentage"]]
#display
age_breakdown

,Player Count,Percentage
Age Group,,
Below 10,17,2.95%
10-14,22,3.82%
15-19,107,18.6%
20-24,258,44.8%
25-29,77,13.4%
30-34,52,9.03%
35-39,31,5.38%
40-44,11,1.91%
45-49,1,0.174%


## Purchasing Analysis - Age

In [60]:
# calculating aggregate purchasing metrics by age group
age_mean = pd.DataFrame(age_agg.mean())
age_sum = pd.DataFrame(age_agg.sum())
age_purchases = pd.DataFrame(age_count.loc[:,'Purchase ID']).rename(columns = {"Purchase ID":"Purchase Count"})
age_avgprice = pd.DataFrame(age_mean.loc[:,'Price']).rename(columns = {"Price":"Average Purchase Value"})
age_tot = pd.DataFrame(age_sum.loc[:,'Price']).rename(columns = {"Price":"Total Purchase Value"})
age_avgprice["Average Purchase Value Per Person"] = pd.DataFrame(age_tot["Total Purchase Value"]/age_breakdown["Player Count"])

# formatting to dollars
age_tot["Total Purchase Value"] = age_tot["Total Purchase Value"].map("${:.6}".format)
age_avgprice["Average Purchase Value"] = age_avgprice["Average Purchase Value"].map("${:.3}".format)
age_avgprice["Average Purchase Value Per Person"] = age_avgprice["Average Purchase Value Per Person"].map("${:.3}".format)

# combining metrics into formatted data frame
age_join = age_purchases.merge(age_avgprice, on = 'Age Group').merge(age_tot, on = "Age Group")

# display
age_join

,Purchase Count,Average Purchase Value,Average Purchase Value Per Person,Total Purchase Value
Age Group,,,,
Below 10,23,$3.35,$4.54,$77.13
10-14,28,$2.96,$3.76,$82.78
15-19,136,$3.04,$3.86,$412.89
20-24,365,$3.05,$4.32,$1114.06
25-29,101,$2.9,$3.81,$293.0
30-34,73,$2.93,$4.12,$214.0
35-39,41,$3.6,$4.76,$147.67
40-44,12,$3.04,$3.32,$36.54
45-49,1,$1.7,$1.7,$1.7


## Top Spenders

In [62]:
# Create Player level aggregate
player_agg = purchase_df.groupby('SN')

# Calculate aggregate spending metrics at the player level
player_count = player_agg.count()
player_tot = player_agg.sum().sort_values("Price", ascending = False)
player_avgprice = player_agg.mean()

# Combining aggregate metrics
player_join = player_tot.merge(player_avgprice,on = 'SN', suffixes = ['_tot','_avg'])

# Sorting Results by Total Spend, desccending
player_join2 = player_join.merge(player_count, on = 'SN').sort_values("Price_tot", ascending = False)

# Selecting the top five spenders
top_spenders = player_join2[["Purchase ID","Price_avg",'Price_tot']].iloc[[0,1,2,3,4],:].rename(columns = {"Purchase ID":"Purchase Count","Price_avg":"Average Purchase Value","Price_tot":"Total Spend"})

# formatting dollars
top_spenders["Average Purchase Value"] = top_spenders["Average Purchase Value"].map("${:.3}".format)
top_spenders["Total Spend"] = top_spenders["Total Spend"].map("${:.4}".format)

# display
top_spenders

,Purchase Count,Average Purchase Value,Total Spend
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.4,$13.62
Iskadarya95,3,$4.37,$13.1


## Most Popular Items

In [16]:
# Creating aggregrate at the Item level
item_agg = purchase_df.groupby('Item ID')

# Calculating Aggregate metrics at the item level 
item_name = pd.DataFrame(item_agg["Item Name"].max()) # gets name and id. 
item_price = pd.DataFrame(item_agg["Price"].mean()) # gets id and price
item_count = pd.DataFrame(item_agg["Purchase ID"].count()) # id and purchase count
item_tot = pd.DataFrame(item_agg['Price'].sum()) # id and total sales.

# combining aggregate results
item_join = item_name.merge(item_count, on = 'Item ID',how = 'inner').merge(item_tot,on = 'Item ID').merge(item_price, on = 'Item ID')

# Sorting results by count of purchase id, descending, renaming fields
item_join = item_join.sort_values("Purchase ID",ascending = False)
item_join = item_join.rename(columns = {"Purchase ID":"Purchase Count","Price_x":"Total Sales","Price_y":"Average Price"})

# selecting top five results
item_pop = item_join.iloc[[0,1,2,3,4],:]

# formatting dollars
item_pop["Average Price"] = item_pop["Average Price"].map("${:.3}".format)
item_pop["Total Sales"] = item_pop["Total Sales"].map("${:.4}".format)

# display
item_pop

C:\Users\msflo\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\msflo\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Item Name,Purchase Count,Total Sales,Average Price
Item ID,,,,
92,Final Critic,13,$59.99,$4.61
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
145,Fiery Glass Crusader,9,$41.22,$4.58
132,Persuasion,9,$28.99,$3.22
108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$3.53


## Most Profitable Items

In [63]:
# sorting previously created aggregate results by Total item sales, descending, 
# Selecting top five results
item_prof = item_join.sort_values("Total Sales", ascending = False).iloc[[0,1,2,3,4],:]

# formatting dollars
item_prof["Average Price"] = item_prof["Average Price"].map("${:.3}".format)
item_prof["Total Sales"] = item_prof["Total Sales"].map("${:.4}".format)

# display
item_prof

,Item Name,Purchase Count,Total Sales,Average Price
Item ID,,,,
92,Final Critic,13,$59.99,$4.61
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
82,Nirvana,9,$44.1,$4.9
145,Fiery Glass Crusader,9,$41.22,$4.58
103,Singed Scalpel,8,$34.8,$4.35
